# Automatic Differentiation with `torch.autograd`
在训练神经网络时，最常使用的算法是 **back propagation**。在该算法中，parameters (model weights)会根据 **loss function** 相对于给定parameter 的 **gradient** 进行调整。

为了计算这些 **gradient**，PyTorch 有一个内置的求导引擎，名为 `torch.autograd`。它支持对任何计算图自动进行梯度计算。

考虑最简单的单隐层神经网络，它有输入 `x`、参数 `w` 和 `b`，以及某个`loss function`。可以用以下方式在 PyTorch 中定义它。

In [ ]:
import torch  # 导入PyTorch深度学习框架

# 创建输入张量，维度为5，表示有5个输入特征
x = torch.ones(5)  # input tensor

# 创建目标输出张量，维度为3，表示期望的3个输出值
y = torch.zeros(3)  # expected output

# 创建权重矩阵，维度为5×3，requires_grad=True表示需要计算梯度
w = torch.randn(5, 3, requires_grad=True)

# 创建偏置向量，维度为3，requires_grad=True表示需要计算梯度
b = torch.randn(3, requires_grad=True)

# 线性变换：输入乘以权重矩阵再加上偏置，得到线性输出
z = torch.matmul(x, w)+b

# 计算损失函数：使用带logits的二元交叉熵
# 该函数会自动对z应用sigmoid激活，然后计算与目标值y的交叉熵
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensors, Functions and Computational graph
This code defines the following **computational graph**:

![comp-graph](https://docs.pytorch.org/tutorials/_images/comp-graph.png)

在这个网络中，`w` 和 `b` 是我们需要优化的**parameters**。因此，我们需要能够计算 `loss function` 关于这些变量的梯度。为了做到这一点，我们设置这些张量的 `requires_grad` 属性。

>**Note**
>You can set the value of `requires_grad` when creating a tensor, or later by using `x.requires_grad_(True)` method.


我们应用于张量以构建计算图的函数实际上是Function类的一个对象。这个对象知道如何在`forward` direction时计算函数值，并且也知道在 `backward propagation` 步骤中如何计算其导数(derivative)。对反向传播函数的引用存储在张量的`grad_fn`属性中。你可以在[文档中找到关于Function的更多信息](https://pytorch.org/docs/stable/autograd.html#function)。

具体来说，在深度学习计算图构建过程中，操作张量的函数不是普通函数，而是Function类的实例。正向传播就是正常计算函数值的过程，而反向传播则是计算导数以进行梯度下降优化的过程。`grad_fn`属性则记录了反向传播时计算导数的函数引用，通过这个属性，计算图可以在反向传播时顺利计算梯度。

- 例如，在 PyTorch 框架中，当执行tensor1 + tensor2这样的操作时，会创建一个AddBackward类（继承自Function）的对象来处理反向传播计算加法操作的导数，这个对象的引用就会存储在结果张量的grad_fn属性中 。

In [ ]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")